In [0]:

data = [
    (1, "Ava Johnson", "US", "Free", "2024-01-01", "9999-12-31", True),
    (2, "Liam Patel", "UK", "Premium", "2024-01-01", "9999-12-31", True),
    (3, "Sofia Chen", "IN", "Free", "2024-01-01", "9999-12-31", True),
]
cols = ["user_id", "user_name", "region", "plan_type", "effective_start_date", "effective_end_date", "is_current"]

df = spark.createDataFrame(data,cols)
df.write.format('delta').mode('overwrite').saveAsTable('scd_type2_implementation_spotify')
df = spark.table('scd_type2_implementation_spotify')
df.show()

In [0]:
incoming_data = [
    (1, "Ava Johnson", "US", "Premium"),
    (2, "Liam Patel", "UK", "Premium"),
    (3, "Sofia Chen", "IN", "Free"),
    (4, "Noah Kim", "AU", "Free"),
]
df_new = spark.createDataFrame(incoming_data, ["user_id","user_name","region","plan_type"])
df_new.show()

In [0]:
from pyspark.sql.functions import current_date, lit, col

df.alias('df_actual_table').merge(
    df_new.alias('df_new_table'),
    "df_actual_table.user_id = df_new_table.user_id AND df_actual_table.plan_type <> df_new_table.plan_type"
).whenMatchedUpdate(
    set = {
        "effective_end_date": current_date(),
        "is_current": lit(False)
    }
).whenNotMatchedInsert(
    values = {
        "user_id": col("df_new_table.user_id")
    }
).execute()
